In [9]:
import pandas as pd
import numpy as np

In [10]:
수익성 = pd.read_csv('./datasets/코스닥/코스닥 수익성.csv',encoding='cp949')
안정성 = pd.read_csv('./datasets/코스닥/코스닥 안정성.csv',encoding='cp949')
활동성 = pd.read_csv('./datasets/코스닥/코스닥 활동성.csv',encoding='cp949')
성장성 = pd.read_csv('./datasets/코스닥/코스닥 성장성 계산용.csv',encoding='cp949')
현금흐름 = pd.read_csv('./datasets/코스닥/코스닥 현금흐름 + 누적수익성비율.csv',encoding='cp949')
연구개발 = pd.read_csv('./datasets/코스닥/코스닥 rs.csv',encoding='cp949')
EBIT = pd.read_csv('./datasets/코스닥/에비~ 코스닥.csv',encoding='cp949')
per_pbr_총부채 = pd.read_csv('./datasets/코스닥/per pbr 총부채 코스닥.csv',encoding='cp949')
직원수_업력 = pd.read_csv('./datasets/코스닥/코스닥 직원수 급여 업력 외국인 대주주.csv',encoding='cp949')
시가총액 = pd.read_csv('./datasets/코스닥/코스닥시가총액.csv',index_col=0)  #### 시총도 붙여야됨
단기차입금 = pd.read_csv('./datasets/코스닥/코스닥 단기차입금 유동성 장기부채.csv',encoding='cp949')
거시변수 = pd.read_excel('./datasets/거시경제 변수/거시변수.xlsx')
상장일 = pd.read_csv('./datasets/코스닥/코스닥 상장일.csv',encoding='cp949')

In [11]:
df = 안정성.merge(수익성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(활동성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(성장성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(현금흐름,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(연구개발,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(EBIT,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(per_pbr_총부채,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(직원수_업력,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(단기차입금,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(상장일,on=['회사명','거래소코드','회계년도'],how='left')

In [12]:
## 거래소코드 패딩
def padding(x):
    x = str(x)
    return x.rjust(6, "0")
## 외국계기업 제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

# 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

def func_not(x):
    return not x

def change_year_2(x):
    return (x[:4])

In [13]:
## 거래소 코드 패딩
df['거래소코드'] = df['거래소코드'].map(padding)
# 회계년도 제거
df['회계년도'] = df['회계년도'].map(change_year)

In [14]:
## 시가총액 merge
시가총액['회계년도'] = 시가총액['회계년도'].astype(str).str[:4].astype(int)
df = df.merge(시가총액,on=['거래소코드','회계년도'],how='left')
df['시가총액(백만원)'] = df['시가총액'] / 1000000
df.drop(['시가총액'],axis=1,inplace=True)

## 거시변수 merge
거시변수.rename(columns={'회계연도':'회계년도'},inplace=True)
df = df.merge(거시변수,on=['회계년도'],how='left')

In [15]:
## 중복되는 데이터 제거
df.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df.reset_index(drop=True, inplace=True)

## 없음


In [16]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 15617 개
전체 기업 수 1744


In [17]:
## 외국계 기업 제거
print("제거할 데이터 수 : ", len(df[df["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
print('제거할 외국계 기업 수',len(df[~df["거래소코드"].map(func_isnt_fcompany)].회사명.unique()))

df = df[df["거래소코드"].map(func_isnt_fcompany)]
print('남은 데이터 수',df.shape[0]) # 외국계기업 데이터 24개 제거
print('남은 기업 수',len(df['회사명'].unique()))

제거할 데이터 수 :  152
제거할 외국계 기업 수 23
남은 데이터 수 15465
남은 기업 수 1721


In [18]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df["회사명"].unique()))
print("거래소코드수 : ", len(df["거래소코드"].unique()))

회사수 :  1721
거래소코드수 :  1743


In [19]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(df["회사명"].unique()):
    if len(df[df["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수", len(list_overlap_code), "개")

거래소코드가 2개인 기업 수 22 개


In [20]:
list_overlap_code

['(주)본느',
 '(주)세화피앤씨',
 '(주)아이엘사이언스',
 '(주)알로이스',
 '(주)에스에이티',
 '(주)엔케이맥스',
 '(주)원바이오젠',
 '(주)인산가',
 '(주)자비스',
 '(주)정다운',
 '(주)카이노스메드',
 '(주)티에스트릴리온',
 '(주)포인트엔지니어링',
 '(주)한국비엔씨',
 '(주)휴럼',
 '나무기술(주)',
 '소프트캠프(주)',
 '씨아이에스(주)',
 '에스케이오션플랜트(주)',
 '엠에프엠코리아(주)',
 '줌인터넷(주)',
 '패션플랫폼(주)']

In [21]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df[df["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df[df["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)

In [22]:
# 회사명이 2개인 거래소코드 존재 확인 
list_overlap_code = list()
for i in list(df["거래소코드"].unique()):
    if len(df[df["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[]

In [23]:
## 연결 결측치 개별로 채우기
for i in range(df.shape[0]):
    if pd.isnull(df.loc[i,'유보율(IFRS연결)(백만원)']):
       df.loc[i,'유보율(IFRS연결)(백만원)'] = df.loc[i,'유보율(IFRS)']
    if pd.isnull(df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'자산(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'부채(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]자본(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'당기순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융원가(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융원가(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융원가(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융수익(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융수익(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융수익(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)'] = df.loc[i,'매출액(수익)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'연구개발비(IFRS연결)(천원)']):
       df.loc[i,'연구개발비(IFRS연결)(천원)'] = df.loc[i,'연구개발비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      유형자산(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자본금회전률(IFRS연결)']):
       df.loc[i,'자본금회전률(IFRS연결)'] = df.loc[i,'자본금회전률(IFRS)']
    if pd.isnull(df.loc[i,'총자본회전률(IFRS연결)']):
       df.loc[i,'총자본회전률(IFRS연결)'] = df.loc[i,'총자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'재고자산회전률(IFRS연결)']):
       df.loc[i,'재고자산회전률(IFRS연결)'] = df.loc[i,'재고자산회전률(IFRS)']
    if pd.isnull(df.loc[i,'유형자산회전율(IFRS연결)']):
       df.loc[i,'유형자산회전율(IFRS연결)'] = df.loc[i,'유형자산회전율(IFRS)']
    if pd.isnull(df.loc[i,'자기자본회전률(IFRS연결)']):
       df.loc[i,'자기자본회전률(IFRS연결)'] = df.loc[i,'자기자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'타인자본회전률(IFRS연결)']):
       df.loc[i,'타인자본회전률(IFRS연결)'] = df.loc[i,'타인자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'경영자본순이익률(IFRS연결)']):
       df.loc[i,'경영자본순이익률(IFRS연결)'] = df.loc[i,'경영자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'금융비용부담률(IFRS연결)']):
       df.loc[i,'금융비용부담률(IFRS연결)'] = df.loc[i,'금융비용부담률(IFRS)']
    if pd.isnull(df.loc[i,'총자본순이익률(IFRS연결)']):
       df.loc[i,'총자본순이익률(IFRS연결)'] = df.loc[i,'총자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'매출액총이익률(IFRS연결)']):
       df.loc[i,'매출액총이익률(IFRS연결)'] = df.loc[i,'매출액총이익률(IFRS)']
    if pd.isnull(df.loc[i,'자기자본순이익률(IFRS연결)']):
       df.loc[i,'자기자본순이익률(IFRS연결)'] = df.loc[i,'자기자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'총자본정상영업이익률(IFRS연결)']):
       df.loc[i,'총자본정상영업이익률(IFRS연결)'] = df.loc[i,'총자본정상영업이익률(IFRS)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)'] = df.loc[i,'자산(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'자본(*)(요약)(IFRS연결)']):
       df.loc[i,'자본(*)(요약)(IFRS연결)'] = df.loc[i,'자본(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'현금비율(IFRS연결)(백만원)']):
       df.loc[i,'현금비율(IFRS연결)(백만원)'] = df.loc[i,'현금비율(IFRS)']
    if pd.isnull(df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'차입금의존도(IFRS연결)']):
       df.loc[i,'차입금의존도(IFRS연결)'] = df.loc[i,'차입금의존도(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'감가상각비(IFRS연결)(천원)']):
       df.loc[i,'감가상각비(IFRS연결)(천원)'] = df.loc[i,'감가상각비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'부채비율(IFRS연결)']):
       df.loc[i,'부채비율(IFRS연결)'] = df.loc[i,'부채비율(IFRS)']
    if pd.isnull(df.loc[i,'당좌비율(IFRS연결)']):
       df.loc[i,'당좌비율(IFRS연결)'] = df.loc[i,'당좌비율(IFRS)']
    if pd.isnull(df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)']):
       df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)'] = df.loc[i,'*이자비용(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'순운전자본비율(IFRS연결)']):
       df.loc[i,'순운전자본비율(IFRS연결)'] = df.loc[i,'순운전자본비율(IFRS)']
    if pd.isnull(df.loc[i,'유동성장기부채(요약)(IFRS연결)(백만원)']):
       df.loc[i,'유동성장기부채(요약)(IFRS연결)(백만원)'] = df.loc[i,'유동성장기부채(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'단기차입금(요약)(IFRS연결)(백만원)']):
       df.loc[i,'단기차입금(요약)(IFRS연결)(백만원)'] = df.loc[i,'단기차입금(요약)(IFRS연결)(백만원)']


In [25]:
drop_col = ['유보율(IFRS)' ,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)','[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)',
'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS)(백만원)', 
'부채(*)(요약)(IFRS)(백만원)','[제조]자본(*)(요약)(IFRS)(백만원)','당기순이익(손실)(요약)(IFRS)(백만원)', 
'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)','금융원가(요약)(IFRS)(백만원)','금융수익(요약)(IFRS)(백만원)',
'매출액(수익)(요약)(IFRS)(백만원)','연구개발비(IFRS)(천원)',
'[제조]      유형자산(요약)(IFRS)(백만원)','자본금회전률(IFRS)',
'총자본회전률(IFRS)','재고자산회전률(IFRS)','유형자산회전율(IFRS)','자기자본회전률(IFRS)','타인자본회전률(IFRS)',
'경영자본순이익률(IFRS)','금융비용부담률(IFRS)','총자본순이익률(IFRS)','매출액총이익률(IFRS)',
'자기자본순이익률(IFRS)','총자본정상영업이익률(IFRS)','자산(*)(요약)(IFRS)','자본(*)(요약)(IFRS)',
 '현금비율(IFRS)','* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)',
 '차입금의존도(IFRS)(백만원)','감가상각비(IFRS)(천원)','부채비율(IFRS)',
'당좌비율(IFRS)','*이자비용(요약)(IFRS)(백만원)','순운전자본비율(IFRS)',
'[제조]자산(*)(요약)(IFRS)(백만원)_x','[제조]자산(*)(요약)(IFRS연결)(백만원)_x',
'[제조]자산(*)(요약)(IFRS)(백만원)_y','[제조]자산(*)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_y', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_x', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_x',
'유동성장기부채(요약)(IFRS)(백만원)','단기차입금(요약)(IFRS)(백만원)']

In [26]:
df.drop(drop_col,axis=1,inplace=True)

In [27]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자산(*)(요약)(IFRS연결)', '자본(*)(요약)(IFRS연결)',
       '차입금의존도(IFRS연결)', '유보율(IFRS연결)(백만원)', '현금비율(IFRS연결)(백만원)',
       '부채비율(IFRS연결)', '당좌비율(IFRS연결)', '순운전자본비율(IFRS연결)',
       '*이자비용(요약)(IFRS연결)(백만원)', '* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)',
       '감가상각비(IFRS연결)(천원)', 'EBITDA(IFRS)(백만원)', '총자본정상영업이익률(IFRS연결)',
       '자기자본순이익률(IFRS연결)', '총자본순이익률(IFRS연결)', '매출액총이익률(IFRS연결)',
       '금융비용부담률(IFRS연결)', '경영자본순이익률(IFRS연결)', '자본금회전률(IFRS연결)',
       '유형자산회전율(IFRS연결)', '재고자산회전률(IFRS연결)', '자기자본회전률(IFRS연결)',
       '타인자본회전률(IFRS연결)', '총자본회전률(IFRS연결)', '[제조]      유형자산(요약)(IFRS연결)(백만원)',
       '[제조]자본(*)(요약)(IFRS연결)(백만원)', '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)',
       '[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)',
       '[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)',
       '매출액(수익)(요약)(IFRS연결)(백만원)', '연구개발비(IFRS연결)(천원)',
       '금융원가(요약)(IFRS연결)(백만원)', '금융수익(요약)(IFRS연결)(백만원)',
       '법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)', '당기순이익(손실)(요약)(IFRS연결)(백만원)',
       '자산(*)(요약)(IFRS

In [28]:
## 성장성 지표인 증가율 컬럼 만들기
성장성_col = ['[제조]      유형자산(요약)(IFRS연결)(백만원)','매출액(수익)(요약)(IFRS연결)(백만원)',
           '[제조]자본(*)(요약)(IFRS연결)(백만원)','자산(*)(요약)(IFRS연결)',
           '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']

In [29]:
## 증가율 컬럼 전기에 음수일때 당기에 음수지만 증가할때 증가율 계산
## ex (2011년 -20, 2012년 -10) -> -10-(-20) / np.abs(-20)= 
증가율_df = (df.groupby(['회사명', '거래소코드'])[성장성_col].shift(0) - df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1))
증가율_df.rename(columns={'[제조]      유형자산(요약)(IFRS연결)(백만원)':'유형자산증가율', '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액증가율',
       '[제조]자본(*)(요약)(IFRS연결)(백만원)':'자기자본증가율', '자산(*)(요약)(IFRS연결)':'총자본증가율',
       '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'정상영업이익증가율'},inplace=True)

df.drop(['[제조]      유형자산(요약)(IFRS연결)(백만원)','[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'],axis=1,inplace=True)
df = pd.concat([df,증가율_df],axis=1)

In [30]:
## 기업별 업력 설립일 이후 지속된 기간
df['업력'] = df['회계년도'] -  pd.to_datetime(df['설립일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)
df.drop('설립일',axis=1,inplace=True)

In [31]:
## 누적수익성비율
df['누적수익성비율'] = df['[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']/df['자산(*)(요약)(IFRS연결)']

## 이자부담율
df['이자부담률'] = df['EBITDA(IFRS)(백만원)'] / df['*이자비용(요약)(IFRS연결)(백만원)'] 

## 부채상환계수
df['부채상환계수'] = df['EBITDA(IFRS)(백만원)'] / (df['*이자비용(요약)(IFRS연결)(백만원)']+df['단기차입금(요약)(IFRS연결)(백만원)'] + df['유동성장기부채(요약)(IFRS연결)(백만원)'])

## 현금흐름_자산 = 영업현금흐름 / 전기총자산
df['현금흐름 대 자산'] = df.groupby(['회사명','거래소코드'])['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'].shift(0) / df.groupby(['회사명','거래소코드'])['자산(*)(요약)(IFRS연결)'].shift(1)

## 현금흐름_매출액
df['현금흐름 대 매출액'] = df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

# 토빈Q
df['토빈Q'] = (df['부채(*)(요약)(IFRS연결)(백만원)'] + df['시가총액(백만원)']) / df['자산(*)(요약)(IFRS연결)']

## TMD
df['TMD'] = df['부채(*)(요약)(IFRS연결)(백만원)'] / df['시가총액(백만원)']

## 알트만 변수
df['시총/총자산'] = df['시가총액(백만원)'] / df['자산(*)(요약)(IFRS연결)']

## RD -단위도 수정
df['연구개발비(IFRS연결)(백만원)'] = df['연구개발비(IFRS연결)(천원)']/1000
df.drop(['연구개발비(IFRS연결)(천원)'],axis=1,inplace=True)
df['RD'] = df['연구개발비(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

## 직원수변동률 , 직원인건비총액(백만원단위)
df['직원수변동률%'] = df.groupby(['회사명','거래소코드'])['합계_기말인원(명)'].pct_change()*100
df['직원인건비총액(백만원)'] = (df['합계_년간총급여(천원)']/1000) * df['합계_기말인원(명)']
df.drop(['합계_년간총급여(천원)','합계_기말인원(명)'],axis=1,inplace=True)

## 알트만 변수  EBIT = 법인세비용차감전순이익+금융원가-금융수익
df['EBIT/총자산'] = (df['법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] + df['금융원가(요약)(IFRS연결)(백만원)']-df['금융수익(요약)(IFRS연결)(백만원)'])/df['자산(*)(요약)(IFRS연결)']

## PBR , PER
df['PBR'] = df['시가총액(백만원)'] / df['당기순이익(손실)(요약)(IFRS연결)(백만원)']
df['PER'] = df['시가총액(백만원)']  / df['자산(*)(요약)(IFRS연결)']

In [32]:
rename_list = {'자산(*)(요약)(IFRS연결)':'자산', '자본(*)(요약)(IFRS연결)':'자본',
       '차입금의존도(IFRS연결)':'차입금의존도', '유보율(IFRS연결)(백만원)':'유보율', '현금비율(IFRS연결)(백만원)':'현금비율',
       '부채비율(IFRS연결)':'부채비율', '당좌비율(IFRS연결)':'당좌비율', '순운전자본비율(IFRS연결)':'순운전자본비율',
       '*이자비용(요약)(IFRS연결)(백만원)':'이자비용', '* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'영업손익',
       '감가상각비(IFRS연결)(천원)':'감가상각비', 'EBITDA(IFRS)(백만원)':'EBITDA', '총자본정상영업이익률(IFRS연결)':'총자본정상영업이익률',
       '자기자본순이익률(IFRS연결)':'자기자본순이익률', '총자본순이익률(IFRS연결)':'총자본순이익률', '매출액총이익률(IFRS연결)':'매출액총이익률',
       '금융손익비율(IFRS연결)':'금융손익비율', '경영자본순이익률(IFRS연결)':'경영자본순이익률', '자본금회전률(IFRS연결)':'자본금회전률',
       '유형자산회전율(IFRS연결)':'유형자산회전율', '재고자산회전률(IFRS연결)':'재고자산회전률', '자기자본회전률(IFRS연결)':'자기자본회전률',
       '타인자본회전률(IFRS연결)':'타인자본회전률', '총자본회전률(IFRS연결)':'총자본회전률', '[제조]자본(*)(요약)(IFRS연결)(백만원)':'자본',
       '[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)':'이익잉여금',
       '[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)':'영업활동으로 인한 현금흐름',
       '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액', '금융원가(요약)(IFRS연결)(백만원)':'금융원가',
       '금융수익(요약)(IFRS연결)(백만원)':'금융수익', '법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)':'법인세비용차감전순이익',
       '당기순이익(손실)(요약)(IFRS연결)(백만원)':'당기순이익', '자산(*)(요약)(IFRS연결)(백만원)':'자산',
       '부채(*)(요약)(IFRS연결)(백만원)':'부채', '외국인_주식분포비율':'외국인지분율', '보통주식비율':'대주주지분율',
       '단기차입금(요약)(IFRS연결)(백만원)':'단기차입금', '[제조]      유동성장기부채(요약)(IFRS연결)':'유동성장기부채',
       '시가총액(백만원)':'시가총액','연구개발비(IFRS연결)(백만원)':'연구개발비','직원인건비총액(백만원)':'직원인건비총액'}

df.rename(columns=rename_list,inplace=True)

In [36]:
df['상장일'] = pd.to_datetime(df['상장일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)
df = df[df['회계년도']>2011].reset_index(drop=True)

In [37]:
pd.set_option('display.max_rows',None)
df.isna().sum()

회사명                            0
거래소코드                          0
회계년도                           0
자산                          1456
자본                          1456
차입금의존도                      1641
유보율                         1641
현금비율                        1641
부채비율                        1641
당좌비율                        1641
순운전자본비율                     1641
이자비용                        1456
영업손익                        1456
감가상각비                       1456
EBITDA                      1643
총자본정상영업이익률                  1641
자기자본순이익률                    1641
총자본순이익률                     1641
매출액총이익률                     1641
금융비용부담률(IFRS연결)             1641
경영자본순이익률                    1641
자본금회전률                      1641
유형자산회전율                     1641
재고자산회전률                     1641
자기자본회전률                     1641
타인자본회전률                     1641
총자본회전률                      1641
자본                          1456
이익잉여금                       1456
영업활동으로 인한 현금흐름              1456
매출액       

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14077 entries, 0 to 14076
Data columns (total 66 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   회사명                       14077 non-null  object 
 1   거래소코드                     14077 non-null  object 
 2   회계년도                      14077 non-null  int64  
 3   자산                        12621 non-null  float64
 4   자본                        12621 non-null  float64
 5   차입금의존도                    12436 non-null  float64
 6   유보율                       12436 non-null  float64
 7   현금비율                      12436 non-null  float64
 8   부채비율                      12436 non-null  float64
 9   당좌비율                      12436 non-null  float64
 10  순운전자본비율                   12436 non-null  float64
 11  이자비용                      12621 non-null  float64
 12  영업손익                      12621 non-null  float64
 13  감가상각비                     12621 non-null  float64
 14  EBITDA

-----

-----

In [40]:
df[df['거래소코드']=='189700']

,회사명,거래소코드,회계년도,자산,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,...,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PBR,PER
1408,(주)디피앤케이,189700,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1409,(주)디피앤케이,189700,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1410,(주)디피앤케이,189700,2014,9950.0,5006.0,34.48,111.24,0.71,98.78,115.79,...,NaN,NaN,NaN,190.433,0.015276,NaN,6895.098,0.176583,NaN,NaN
1411,(주)디피앤케이,189700,2015,17343.0,5305.0,56.88,45.63,3.32,226.93,73.24,...,NaN,NaN,NaN,218.307,0.016674,4.761905,9466.490,-0.060889,NaN,NaN
1412,(주)디피앤케이,189700,2016,13443.0,908.0,63.70,-75.06,0.37,1379.80,79.16,...,NaN,NaN,NaN,207.013,0.012446,-9.090909,8609.240,-0.263111,NaN,NaN
